In [3]:
import selenium
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
import time

class Scraper():

    def __init__(self):
        self.driver = Chrome('./chromedriver')
        url = 'https://www.zoopla.co.uk/to-rent/property/liverpool/?beds_min=1&page_size=25&price_frequency=per_month&price_max=700&view_type=list&q=Liverpool%2C%20Merseyside&radius=0&results_sort=newest_listings&search_source=refine'
        # Scraper Class to Navigate Website
        self.driver.get(url)
        time.sleep(3)
        self.full_prop_list = []
        self.pages = 0

# Look for cookies button and accept cookies
    def load_and_accept_cookies(self):
        try:
            self.driver.switch_to.frame('gdpr-consent-notice') # This is the id of the frame
            accept_cookies_button = self.driver.find_element_by_xpath('//*[@id="save"]')
            accept_cookies_button.click()
            time.sleep(1)
        except AttributeError: # If you have the latest version of selenium, the code above won't run because the "switch_to_frame" is deprecated
            self.driver.switch_to_frame('gdpr-consent-notice') # This is the id of the frame
            accept_cookies_button = self.driver.find_element_by_xpath('//*[@id="save"]')
            accept_cookies_button.click()
            time.sleep(1)

        except:
            pass

    def decide_page(self):
        if self.num_prop < 24:
            self.scrape_webpage()
        elif self.num_prop == 24:
            self.pages +=1
            time.sleep(10)
            if self.pages < 5:
                next_page_button = self.driver.find_element_by_xpath('//a[@class="eaoxhri5 css-xtzp5a-ButtonLink-Button-StyledPaginationLink eaqu47p1"]')
                time.sleep(10)
                self.driver = self.scrape_webpage()
        else:
            print(self.full_prop_list)

    def scrape_webpage(self):
        self.prop_container = self.driver.find_element_by_xpath('//div[@class="css-1anhqz4-ListingsContainer e1b8efd72"]')
        self.prop_list = self.prop_container.find_elements_by_xpath('./div')
        self.num_prop = len(self.prop_list)
        
        for i in range(self.num_prop):
            self.prop_container = self.driver.find_element_by_xpath('//div[@class="css-1anhqz4-ListingsContainer e1b8efd72"]')
            house = self.prop_container.find_elements_by_xpath('./div')[i]
            self.data = {i: {"rent_price": [], "weekly_price": [],"bed_and_bath_num": [], "description": [], "address": []}}
            house.click()
            time.sleep(20) # To make sure the webpage has time to load
            
                # Try to find each corresponding information and append 'None' if not present
            try:
                rent_price = self.driver.find_element_by_xpath('//p[@data-testid = "price"]').text
                self.data[i]['rent_price'].append(rent_price)
            except NoSuchElementException:
                self.data[i]['rent_price'].append('None')

            try:
                weekly_price = self.driver.find_element_by_xpath('//p[@data-testid="rentalfrequency-and-floorareaunit"]').text
                self.data[i]['weekly_price'].append(weekly_price)
            except NoSuchElementException:
                self.data[i]['weekly_price'].append('None')

            try:
                bed_and_bath_num = self.driver.find_element_by_xpath('//div[@class="c-PJLV c-PJLV-iiNveLf-css"]').text
                self.data[i]['bed_and_bath_num'].append(bed_and_bath_num)
            except NoSuchElementException:
                self.data[i]['bed_and_bath_num'].append('None')

                
            try:
                description = self.driver.find_element_by_xpath('//div[@data-testid="listing_description"]').text
                self.data[i]['description'].append(description)
            except NoSuchElementException:
                self.data[i]['description'].append('None')

            try:
                address = self.driver.find_element_by_xpath('//address[@ data-testid="address-label"]').text
                self.data[i]['address'].append(address)
            except NoSuchElementException:
                self.data[i]['address'].append('None')
                
            self.full_prop_list.append(self.data)
            print(self.full_prop_list[i])
            time.sleep(10)
            self.driver.back()
            time.sleep(10)

    def run_scraper(self):
        self.load_and_accept_cookies()
        self.scrape_webpage()
        self.decide_page()

new_Scraper_Website = Scraper()
new_Scraper_Website.run_scraper()

    






{0: {'rent_price': ['£425 pcm'], 'weekly_price': ['£98 pw'], 'bed_and_bath_num': ['1 bed\n1 bath'], 'description': ['Property Reference: 1387063.\n\nWe are proud to offer this delightful 1 bedroom, 1 bathroom flat in a great location.\n\nAvailable to move in from 11 May 2022. Property is offered furnished.\n\nViewing highly recommended. Contact OpenRent today for more details or to arrange a viewing!\n\n\n\nRent £425.00 per month (£98.08 per week)\nDeposit / Bond is £490.38\n\nMinimum tenancy is 12 months\nMaximum number of tenants is 1\n\n\n- Rent Amount: £425.00 per month (£98.08 per week)\n- Deposit / Bond: £490.38\n- 1 Bedrooms\n- 1 Bathrooms\n- Property comes furnished\n- Available to move in from 11 May, 2022\n- Minimum tenancy term is 12 months\n- Maximum number of tenants is 1\n- DSS enquiries welcome\n- Students welcome to enquire\n- Pets considered / by arrangement\n- Smokers considered\n- Family Friendly\n- Bills not included\n- No Parking Available\n- No Garden Access\n- EP

WebDriverException: Message: target frame detached
  (Session info: chrome=101.0.4951.54)
